In [44]:
#PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import requests
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define the API endpoint
api_url = "http://172.16.5.6:8080/v1/web/test12"

# Fetch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Create a pandas DataFrame from the data
    # Assuming the API response is a list of dictionaries
    df = pd.DataFrame(data['data'])
    
else:
    print(f"Failed to fetch data: {response.status_code}")
    
data = df
print(data)

In [46]:
#file_path = "data/datadummy.json"
#with open(file_path,'r') as f:
    #json_data = json.load(f)
#data = pd.DataFrame(json_data)

#data = pd.DataFrame(json_data['data'])
#print(data)

In [47]:
#PARAMETER
#WMA
weights = [0.1, 0.3, 0.6]

#EWMA
alpha_ewma = 0.4

#SES & DES
alpha_ses = 0.65  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.45   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)

In [ ]:
# Get mean and standard deviation of 12 periods before the last one
data['mean_12'] = data['D'].apply(lambda x: np.mean(x[-13:-1]))  # Use 12 periods before the last one
data['std_12'] = data['D'].apply(lambda x: np.std(x[-13:-1]))    # Use 12 periods before the last one

# Get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# Limit the original data to upper bound (using the 12 periods before the last one)
data['clipped_d'] = data.apply(lambda row: np.clip(row['D'][-13:-1], 0, row['ub']).tolist(), axis=1)

# Display the updated DataFrame
display(data)


In [ ]:
# Calculate Simple Moving Average
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

data['ma'] = data['clipped_d'].apply(calculate_ma)
data['ma_result'] = data['ma'].apply(lambda x: x[-1:])
data['ma_result'] = data['clipped_d'].apply(lambda x: np.mean(x))
display(data)


In [ ]:
# Step 2: Clip the last 15 periods of 'D' to this upper bound for WMA
data['wma_clipped_d'] = data.apply(lambda row: np.clip(row['D'][-15:], 0, row['ub']).tolist(), axis=1)

# Step 3: Define the function to compute WMA for 15 periods with 3 NaN
def wma_forecast_current(data, weights):
    wma_values = [None] * 3  # Start with 3 NaN values
    for i in range(3, len(data)):
        # Calculate WMA for periods starting from the 4th (index 3)
        forecast = np.sum(np.array(data[i-3:i]) * weights) / sum(weights)
        wma_values.append(forecast)
    return wma_values, wma_values[-1] if wma_values[-1] is not None else None

# Step 4: Define weights for the last 3 periods (oldest to newest)
weights_wma_3 = [0.1, 0.3, 0.6]

# Step 5: Apply WMA to the 'wma_clipped_d' column
data['wma'], data['wma_result'] = zip(*data['wma_clipped_d'].apply(lambda x: wma_forecast_current(x, weights_wma_3)))

# Display the updated DataFrame
display(data)

In [ ]:
# Calculate Exponential Weighted Moving Average (EWMA)
def ewma(list, alpha = alpha_ewma):
    df = pd.DataFrame(list)
    df['ewma'] = df.ewm(alpha=alpha_ewma, adjust=False).mean()
    return df['ewma'].tolist()

def ewma_forecast(list, alpha):
    ewma_values = ewma(list, alpha)
    if len(ewma_values) > 0:
        # Prediction for the next period
        next_forecast = (1 - alpha) * ewma_values[-1]
    else:
        next_forecast = None
    return ewma_values, next_forecast

data['ewma'], data['ewma_result'] = zip(*data['clipped_d'].apply(lambda x: ewma_forecast(x, alpha_ewma)))

display(data)

In [ ]:
#LINEAR REGRESSION
#  Calculate Linear Regression
def lr(x):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    model =  LinearRegression()
    model.fit(df[['x']], df['y'])
    df.loc[len(df), 'x'] = len(df) + 1
    return model.predict(df[['x']])

data['lr'] = data['clipped_d'].apply(lambda x: lr(x))
data['lr_result'] = data['lr'].apply(lambda x: x[-1:])
display(data)


In [ ]:
#POLYNOMIAL 2ND AND 3RD
# Calculate Polynomial Regression
def pr(x, pr_degree):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)

    X = df[['x']]  # Independent variable (reshape to 2D array)
    y = df['y']    # Dependent variable

    poly = PolynomialFeatures(degree=pr_degree)  # Create polynomial features
    X_poly = poly.fit_transform(X)  # Transform input features
    poly_model = LinearRegression()  # Initialize linear regression model
    poly_model.fit(X_poly, y)  # Fit polynomial model

    df.loc[len(df), 'x'] = len(df) + 1
    X_all_poly = poly.transform(df[['x']])
    return poly_model.predict(X_all_poly)  

data['pr2'] = data['clipped_d'].apply(lambda x: pr(x, 2))
data['pr2_result'] = data['pr2'].apply(lambda x: x[-1:])
data['pr3'] = data['clipped_d'].apply(lambda x: pr(x, 3))
data['pr3_result'] = data['pr3'].apply(lambda x: x[-1:])
display(data)


In [ ]:
#SES
def ses(x, alpha = alpha_ses):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = SimpleExpSmoothing(df['y']).fit(smoothing_level=alpha, optimized=False).fittedvalues
    return new_data.tolist()

data['ses'] = data['clipped_d'].apply(lambda x: ses(x, alpha_ses))
data['ses_result'] = data['ses'].apply(lambda x: x[-1:])
display(data)

In [ ]:
#DES
def des(x, alpha = alpha_ses, beta = beta_des):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = ExponentialSmoothing(df['y'], trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False).fittedvalues
    return new_data.tolist()

data['des'] = data['clipped_d'].apply(lambda x: des(x,alpha_ses, beta_des))
data['des_result'] = data['des'].apply(lambda x: x[-1:])
display(data)

In [ ]:
# calculate R2 Score and RMSE

def metric(x):
    period_length = len(x['clipped_d'])
    df = pd.DataFrame()
    df['period'] = range(1, period_length + 1)
    df['qty'] = x['clipped_d'][:period_length]
    df['ma'] = x['ma'][:period_length]
    df['wma']= x['wma'][-12:]
    df['ewma'] = x['ewma'][:period_length]
    df['lr'] = x['lr'][:period_length]
    df['pr2'] = x['pr2'][:period_length]
    df['pr3'] = x['pr3'][:period_length]
    df['ses'] = x['ses'][:period_length]
    df['des'] = x['des'][:period_length]
    # display(df)
    
    # df = pd.concat([pd.DataFrame(x['lr']), df], axis=1)
    result = []
    for i in df.iloc[:, -7:]:
        rmse = np.sqrt(mean_squared_error(df['qty'], df[i]))  # Calculate RMSE
        r2 = r2_score(df['qty'], df[i])  # Calculate R2
        mae = mean_absolute_error(df['qty'], df[i])
        result.append({'model': i, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
        
    # display(result)
    # df_result = pd.DataFrame()

    
    return result


data['metric'] = data.apply(lambda x: metric(x), axis=1)

# display(data['metric'][1])
display(data)

In [ ]:
# BEST MODEL SELECTION
def best_select(x, key):
    return max(x, key=lambda x: x['R2'])[key]

def best_number(x):
    return x[x['best_model']][-1]

data['best_model'] = data['metric'].apply(lambda x: best_select(x, 'model'))
data['best_r2'] = data['metric'].apply(lambda x: best_select(x, 'R2'))
data['best_value'] = data.apply(lambda x: best_number(x), axis=1)
data['FD'] = np.ceil(data['best_value'])
display(data)

In [71]:
# Get mean and standard deviation of data for periods 12 to 1
data['mean_12_FD'] = data['D'].apply(lambda x: np.mean(x[1:13]))
data['std_12_FD'] = data['D'].apply(lambda x: np.std(x[1:13]))

data['ub_FD'] = data['mean_12_FD'] + 1.5 * data['std_12_FD']

data['clipped_d_FD'] = data.apply(lambda row: np.clip(row['D'][1:13], 0, row['ub_FD']).tolist(), axis=1)

data['best_value_FD'] = data.apply(lambda x: best_number(x), axis=1)
print(data['clipped_d_FD'])
def apply_best_model_forecast(row):
    # Retrieve the best model and relevant data
    best_model = row['best_model']
    data = row['D'][3:15]  # Use periods 12 to 1 for forecasting

    if best_model == 'Moving Average':
        # Moving Average: Take the mean of the data
        forecast = np.mean(data)
    
    elif best_model == 'Exponential Weighted Moving Average':
        # EWMA: Calculate with a decay factor
        alpha = 0.4  # Example smoothing factor
        weights = np.array([(1 - alpha) ** i for i in range(len(data))][::-1])
        forecast = np.sum(weights * data) / np.sum(weights)
    
    elif best_model == 'Weighted Moving Average':
        # WMA: Use predefined weights for 3 periods
        weights = [0.2, 0.3, 0.5]  # Example weights (w1, w2, w3)
        if len(data) >= len(weights):
            forecast = forecast = np.sum(np.array(data[i-3:i]) * weights) / sum(weights)
        else:
            forecast = np.nan  # Not enough data to calculate WMA

    elif best_model == 'Linear Regression':
        # Linear Regression: Fit a simple linear regression and predict
        X = np.arange(len(data)).reshape(-1, 1)  # Time indices
        y = np.array(data)
        coef = np.polyfit(X.flatten(), y, 1)  # Linear regression
        forecast = coef[0] * len(data) + coef[1]  # Predict for next period
    
    elif best_model == 'Polynomial Regression (2nd degree)':
        # Polynomial Regression (2nd degree): Fit and predict
        X = np.arange(len(data)).reshape(-1, 1)  # Time indices
        y = np.array(data)
        coef = np.polyfit(X.flatten(), y, 2)  # Quadratic regression
        forecast = coef[0] * (len(data) ** 2) + coef[1] * len(data) + coef[2]
    
    elif best_model == 'Polynomial Regression (3rd degree)':
        # Polynomial Regression (3rd degree): Fit and predict
        X = np.arange(len(data)).reshape(-1, 1)
        y = np.array(data)
        coef = np.polyfit(X.flatten(), y, 3)  # Cubic regression
        forecast = (
            coef[0] * (len(data) ** 3)
            + coef[1] * (len(data) ** 2)
            + coef[2] * len(data)
            + coef[3]
        )
    
    elif best_model == 'Simple Exponential Smoothing':
        # SES: Use statsmodels to forecast
        model = SimpleExpSmoothing(data).fit(smoothing_level=0.65, optimized=False)
        forecast = model.forecast(1)[0]  # Forecast one step ahead
    
    elif best_model == 'Double Exponential Smoothing':
        # DES: Use statsmodels Holt model
        model = Holt(data).fit(smoothing_level=0.65, smoothing_slope=0.45, optimized=False)
        forecast = model.forecast(1)[0]  # Forecast one step ahead
    
    else:
        # Default case: Return NaN for unknown models
        forecast = np.nan
    
    return forecast

# Apply the function to calculate final forecasts
data['FD_final'] = np.ceil(data['best_value_FD'])

# Display the updated DataFrame
display(data)


0    [105.0, 110.0, 108.0, 115.0, 120.0, 125.0, 130...
1    [210.0, 220.0, 215.0, 230.0, 240.0, 245.0, 250...
2    [55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85....
3    [310.0, 320.0, 315.0, 330.0, 340.0, 345.0, 350...
4    [505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535...
5    [155.0, 160.0, 158.0, 165.0, 170.0, 175.0, 180...
6    [405.0, 410.0, 408.0, 415.0, 420.0, 425.0, 430...
7    [85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 110.0, ...
8    [260.0, 270.0, 265.0, 280.0, 290.0, 295.0, 300...
9    [610.0, 620.0, 615.0, 630.0, 640.0, 645.0, 650...
Name: clipped_d_FD, dtype: object


,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,...,best_value,FD,mean_12_FD,std_12_FD,ub_FD,clipped_d_FD,best_value_FD,FD_final,wma,wma_result
0,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",135.083333,16.839727,160.342923,"[108.0, 115.0, 120.0, 125.0, 130.0, 128.0, 135...","[108.0, 111.5, 114.33333333333333, 117.0, 119....",134.695244,...,170.931978,171.0,125.916667,14.180141,147.186878,"[105.0, 110.0, 108.0, 115.0, 120.0, 125.0, 130...",170.931978,171.0,"[None, None, None, 108.3, 112.4, 117.3, 122.5,...",159.205754
1,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",256.666667,21.245915,288.535539,"[215.0, 230.0, 240.0, 245.0, 250.0, 255.0, 260...","[215.0, 222.5, 228.33333333333334, 232.5, 236....",256.544628,...,301.229203,302.0,244.583333,20.962102,276.026486,"[210.0, 220.0, 215.0, 230.0, 240.0, 245.0, 250...",301.229203,302.0,"[None, None, None, 216.0, 224.5, 234.5, 242.0,...",286.121323
2,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",84.416667,14.952471,106.845373,"[58.0, 65.0, 70.0, 75.0, 80.0, 85.0, 88.0, 90....","[58.0, 61.5, 64.33333333333333, 67.0, 69.6, 72...",84.153781,...,114.460498,115.0,76.083333,13.640982,96.544806,"[55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80.0, 85....",114.460498,115.0,"[None, None, None, 58.3, 62.4, 67.3, 72.5, 77....",105.107224
3,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",359.583333,23.314725,394.555420,"[315.0, 330.0, 340.0, 345.0, 350.0, 360.0, 365...","[315.0, 322.5, 328.3333333333333, 332.5, 336.0...",359.546285,...,405.063793,406.0,346.666667,23.033792,381.217354,"[310.0, 320.0, 315.0, 330.0, 340.0, 345.0, 350...",405.063793,406.0,"[None, None, None, 316.0, 324.5, 334.5, 342.0,...",391.733252
4,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",543.333333,18.521759,571.115972,"[515.0, 520.0, 525.0, 530.0, 535.0, 540.0, 545...","[515.0, 517.5, 520.0, 522.5, 525.0, 527.5, 530...",543.009664,...,578.609175,579.0,532.500000,17.260263,558.390394,"[505.0, 510.0, 515.0, 520.0, 525.0, 530.0, 535...",578.609175,579.0,"[None, None, None, 512.5, 517.5, 522.5, 527.5,...",569.669583
5,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",188.166667,18.783119,216.341345,"[158.0, 165.0, 170.0, 175.0, 180.0, 185.0, 190...","[158.0, 161.5, 164.33333333333334, 167.0, 169....",187.861779,...,224.576339,225.0,178.166667,16.313764,202.637312,"[155.0, 160.0, 158.0, 165.0, 170.0, 175.0, 180...",224.576339,225.0,"[None, None, None, 158.3, 162.4, 167.3, 172.5,...",214.804807
6,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",438.166667,18.783119,466.341345,"[408.0, 415.0, 420.0, 425.0, 430.0, 435.0, 440...","[408.0, 411.5, 414.3333333333333, 417.0, 419.6...",437.861779,...,474.576339,475.0,428.166667,16.313764,452.637312,"[405.0, 410.0, 408.0, 415.0, 420.0, 425.0, 430...",474.576339,475.0,"[None, None, None, 408.29999999999995, 412.4, ...",464.804807
7,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",118.166667,18.783119,146.341345,"[88.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0...","[88.0, 91.5, 94.33333333333333, 97.0, 99.6, 10...",117.861779,...,154.576339,155.0,108.166667,16.313764,132.637312,"[85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 110.0, ...",154.576339,155.0,"[None, None, None, 88.3, 92.4, 97.3, 102.5, 10...",144.804807
8,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",309.583333,23.314725,344.555420,"[265.0, 280.0, 290.0, 295.0, 300.0, 310.0, 315...","[265.0, 272.5, 278.3333333333333, 282.5, 286.0...",309.546285,...,355.063793,356.0,296.666667,23.033792,331.217354,"[260.0, 270.0, 265.0, 280.0, 290.0, 295.0, 300...",355.063793,356.0,"[None, None, None, 266.0, 274.5, 284.5, 292.0,...",341.733252
9,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",659.583333,23.314725,694.555420,"[615.0, 630.0, 640.0, 645.0, 650.0, 660.0, 665...","[615.0, 622.5, 628.3333333333334, 632.5, 636.0...",659.546285,...,705.063793,706.0,646.666667,23.033